# TP 2 : Branch-and-bound applied to a knapsack problem to compar results with TP3

### Récupération des données

In [1]:
function readKnaptxtInstance(filename)
    price=Int64[]
    weight=Int64[]
    KnapCap=Int64[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Tests de sondabilités TA, TO et TR basés sur le modèle linéaire

In [2]:
function testSondability_LP( capacity,  borne, BestProfit, Bestsol)
   
    TA, TO, TR = false, false, false
    
    

    if capacity < 0  # Test d'Admissibilité
        TA = true
        
        # println("TA")
        
    elseif (borne < BestProfit)  # Test d'optimalite
        TO = true
        
        # println("TO")
        
    elseif (capacity == 0) || (borne == BestProfit)  # Test de resolution
        TR = true
        
        # println("TR")
       
        
    else
        
        # println("non sondable")
        
    end
    return TA, TO, TR, Bestsol, BestProfit
end

testSondability_LP (generic function with 1 method)

## Procédure de séparation (branching) et stratégie d'exploration permettant de se placer au prochain noeud à traiter

In [3]:
function separateNodeThenchooseNext_lexicographic_depthfirst!(listobjs, listvals, n)
    # this node is non-sondable. Apply the branching criterion to separate it into two subnodes
    # and choose the child-node at the left

    # lexicographic branching criterion: branch on the 1st object not yet fixed
    i, obj = 1, 0
    while((i <= n) && (obj==0))
        if(!(i in listobjs))
            obj=i
        end
        i+=1
    end

    # println("\nbranch on object ", obj, "\n")

    # depthfirst exploration strategy: the node selected will be the most left of the child-nodes just created
    push!(listobjs,obj) #save the identity of the object selected for branching
    push!(listvals,1.0) #save the node selected, identified by the value assigned to the variable/object chosen
end


function exploreNextNode_depthfirst!(listobjs, listvals, listnodes)
    #this node is sondable, go back to parent node then right child if possible

    stop=false
    #check if we are not at the root node
    if (length(listobjs)>= 1)
        #go back to parent node
        obj=pop!(listobjs)
        theval=pop!(listvals)
        tmp=pop!(listnodes)

        #go to right child if possible, otherwise go back to parent
        while( (theval==0.0) && (length(listobjs)>= 1))
            obj=pop!(listobjs)
            theval=pop!(listvals)
            tmp=pop!(listnodes)
        end
        if theval==1.0
            push!(listobjs,obj)
            push!(listvals,0.0)
        else
            # println("\nFINISHED")
            stop=true
        end
    else
        #the root node was sondable
        # println("\nFINISHED")
        stop=true
    end
    return stop
end

exploreNextNode_depthfirst! (generic function with 1 method)

###  Création de la relaxation linéaire (= modèle associé au noeud 0):

In [4]:
function Bornes(price, weight, capacity, listobjs, listvals, BestProfit, Bestsol, option)
    
    borne = 0
    prix = 0
    max = 0

    if option == 1
        weightAct = 0
        priceAct = 0
        for i in 1:length(listobjs)
            if listvals[i] == 1
                if weightAct + weight[listobjs[i]] <= capacity
                    weightAct += weight[listobjs[i]]
                    priceAct += price[listobjs[i]]
                end
            end 
        end
    
        if priceAct >= BestProfit
            Bestsol = copy(listobjs)
            BestProfit = priceAct
        end
            
        if (length(listobjs) < length(price)) 
            for i in (length(listobjs)+1):length(price)
                r = price[i]/weight[i]
                if (!(i in listobjs)) 
                    if r > max        
                        max = r
                    end     
                end
            end
            borne = prix + max*capacity
        else
            borne = prix
        end
        
    elseif option == 2
        for i in 1:length(listobjs)
            if listvals[i] == 1
                capacity = capacity - weight[listobjs[i]]
                # println("Capacity = ", capacity)
                prix = prix + price[listobjs[i]]
                
            end
        end
    
        deleteat!(price, listobjs)
        deleteat!(weight, listobjs)
    
        if capacity >= 0
            if prix > BestProfit
                BestProfit = prix
                Bestsol = copy(listobjs)
            end
        end
        
        
        # Ratio
        r = price./weight
        listR = [(r[i], weight[i]) for i in 1:length(price)]
        listR = sort(listR, by = x -> x[1], rev=true)
        prix1 = 0

        for i in 1:length(listR)
            if listR[i][2] <= capacity
                prix1 = prix1 + listR[i][1]*listR[i][2]
                capacity = capacity - listR[i][2]
            else
                break
            end
        end

        borne = prix + prix1
    end

return borne, capacity, BestProfit, Bestsol

end

Bornes (generic function with 1 method)

### Boucle principale : résoudre une relaxation, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [5]:
function solveKnapInstance(filename, option)

    price, weight, capacity = readKnaptxtInstance(filename)
    n = length(price)

    #model2, x = createModel_LP(price, weight, capacity)

    #create the structure to memorize the search tree for visualization at the end
    trParentnodes=Int64[] #will store orig node of arc in search tree
    trChildnodes=Int64[] #will store destination node of arc in search tree
    trNamenodes=[] #will store names of nodes in search tree

    #intermediate structure to navigate in the search tree
    listobjs=Int64[]
    listvals=Float64[]
    listnodes=Int64[]

    BestProfit::Float64=-1.0
    Bestsol=Float64[]

    current_node_number::Int64=0
    stop = false

    while(!stop)

        # println("\nNode number ", current_node_number, " : \n---------------\n")

        #Update the graphical tree
        push!(trNamenodes,current_node_number+1) 
        if(length(trNamenodes)>=2)
            push!(trParentnodes,listnodes[end]+1) # +1 because the 1st node is "node 0"
            push!(trChildnodes, current_node_number+1) # +1 because the 1st node is "node 0"
        end
        push!(listnodes, current_node_number)

        
        #create LP of current node
        #updateModele_LP!(model2, x, listobjs, listvals)
        
        #println(model2)
        
        # print("Solve the LP model of the current node to compute its bound: start ...\n\n")

        #status = optimize!(model2)

        
        priceAct = deepcopy(price)
        weightAct = deepcopy(weight)
        capacityAct = capacity
        
        borne, capacityAc, Bestp, BestS = Bornes(priceAct, weightAct, capacityAct, listobjs,listvals, BestProfit, Bestsol, option)

        BestProfit = Bestp

        Bestsol = BestS



        TA, TO, TR = testSondability_LP( capacityAc,  borne, Bestp, BestS)

        is_node_sondable = TA || TO || TR

        #Reset_LP!(model2, x, listobjs)

        if(!is_node_sondable)
            separateNodeThenchooseNext_lexicographic_depthfirst!(listobjs, listvals, length(price))
        else
            stop = exploreNextNode_depthfirst!(listobjs, listvals, listnodes)
        end
        
        #resetAll_LP!(model2, x)

        current_node_number = current_node_number + 1
    end

    # println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x = ", Bestsol)

    return BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes

end


solveKnapInstance (generic function with 1 method)

In [6]:
# Afficher le temps d'exécution de la fonction SolveKnapInstance pour une instance donnée
function GetKnapResolutionTime(filename)
    time = @elapsed solveKnapInstance(filename, 2)
    println("Temps d'exécution : ", time, " secondes")
end

GetKnapResolutionTime (generic function with 1 method)

In [7]:
# Retourne le temps d'éxécution de l'instance almost_strongly_correlated/knapPI_5_50_1000_1_-2096.opb.txt
println("Temps d'exécution de l'instance almost_strongly_correlated/knapPI_5_50_1000_1_-2096.opb.txt :")
GetKnapResolutionTime("InstancesKnapSack/almost_strongly_correlated/knapPI_5_50_1000_1_-2096.opb.txt")

Temps d'exécution de l'instance almost_strongly_correlated/knapPI_5_50_1000_1_-2096.opb.txt :


Temps d'exécution : 0.5458104 secondes


In [8]:
# Retourne le temps d'éxécution de l'instance circle/knapPI_16_20_1000_2_-2296.opb.txt
println("Temps d'exécution de l'instance circle/knapPI_16_20_1000_2_-2296.opb.txt :")
GetKnapResolutionTime("InstancesKnapSack/circle/knapPI_16_20_1000_2_-2296.opb.txt")

Temps d'exécution de l'instance circle/knapPI_16_20_1000_2_-2296.opb.txt :
Temps d'exécution : 0.0024385 secondes


In [9]:
# Retourne le temps d'éxécution de l'instance inverse_strongly_correlated/knapPI_4_50_1000_1_-994.opb.txt
println("Temps d'exécution de l'instance inverse_strongly_correlated/knapPI_4_50_1000_1_-994.opb.txt :")
GetKnapResolutionTime("InstancesKnapSack/inverse_strongly_correlated/knapPI_4_50_1000_1_-994.opb.txt")

Temps d'exécution de l'instance inverse_strongly_correlated/knapPI_4_50_1000_1_-994.opb.txt :
Temps d'exécution : 0.0002626 secondes


In [10]:
# Retourne le temps d'éxécution de l'instance weakly_correlated_span/knapPI_12_20_1000_1_-970.opb.txt
println("Temps d'exécution de l'instance weakly_correlated_span/knapPI_12_20_1000_1_-970.opb.txt :")
GetKnapResolutionTime("InstancesKnapSack/weakly_correlated_span/knapPI_12_20_1000_1_-970.opb.txt")

Temps d'exécution de l'instance weakly_correlated_span/knapPI_12_20_1000_1_-970.opb.txt :
Temps d'exécution : 0.0003638 secondes
